In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../")

%pylab inline
pylab.rcParams['figure.figsize'] = (10, 5)

Populating the interactive namespace from numpy and matplotlib


In [2]:
import torch

In [3]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [4]:
torch.cuda.set_device(0)

In [5]:
import ld_gan

# set parameters

In [6]:
BATCH_SIZE  = 128
LATENT_SIZE = 100
LEARNING_RATE = 0.0002
RAND_SEED = 42

# load data

In [7]:
X, Y = ld_gan.data_proc.data_loader.load_data(-1, resize = 64, verbose = 1)

100%|██████████| 17/17 [00:01<00:00, 12.53it/s]

load data from '/export/home/oblum/projects/ls_gan/data/flowers_17/jpg_64'


# do CUDA

In [8]:
cudnn.benchmark = True

In [9]:
torch.cuda.manual_seed_all(RAND_SEED)

# init networks

In [10]:
gen = ld_gan.models.gen.gen_64(latent_size = 100)
dis = ld_gan.models.dis.dis_64()
enc = ld_gan.models.enc.enc_64(n_features = 100)

# init train ops

In [11]:
train_ops = [
                #ld_gan.train_ops.GanDis(gen, dis, LEARNING_RATE),
                #ld_gan.train_ops.GanGen(gen, dis, LEARNING_RATE),
                ld_gan.train_ops.AutoEnc(enc, gen, LEARNING_RATE),    
            ]
#train_ops = [ld_gan.train_ops.Gan(gen, dis, LEARNING_RATE)]

In [12]:
sampler = ld_gan.sample.generate_rand_noise(X, Y, BATCH_SIZE, LATENT_SIZE)

In [13]:
Z, _, X, _ = sampler.next()

In [14]:
X, Z = ld_gan.data_proc.transformer.np_to_tensor(X, Z)

In [15]:
a = enc(X)

In [16]:
a.size()

torch.Size([128, 100, 1, 1])

In [17]:
b = train_ops[0]

In [18]:
# Z.cpu()
# X.cpu()
# enc.cpu()
# gen.cpu()
# dis.cpu()
# train_ops[0].criterion.cpu()

In [19]:
gen.zero_grad()

In [20]:
enc.zero_grad()

In [21]:
X.size()

torch.Size([128, 3, 64, 64])

In [22]:
z = enc(X)

In [23]:
z.size()

torch.Size([128, 100, 1, 1])

In [24]:
c = gen(z)

In [25]:
b.train(X, Z)

RuntimeError: cuda runtime error (2) : out of memory at /pytorch/torch/lib/THC/generic/THCStorage.cu:66

# train

In [ ]:
trainer = ld_gan.trainer.Trainer(gen, 
                                 dis, 
                                 None, 
                                 train_ops,
                                 sampler,
                                 len(X))

In [ ]:
l = trainer.train()

In [ ]:
fname = "/export/home/oblum/projects/ld_gan/projects/x_test.py/model/dis.pth"

In [ ]:
the_model = torch.load(fname)

In [ ]:
Z, _, X, _ = sampler.next()

In [ ]:
Z, X = ld_gan.data_proc.transformer.np_to_tensor(Z, X)

In [ ]:
the_model(X)